In [1]:
#!pip install pandas numpy nltk matplotlib seaborn wordcloud
%run 1_data_load.ipynb

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import wordninja

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kasim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df_news['Time'] = pd.to_datetime(df_news['Time'], format='%d/%m/%Y')
df_news = df_news.rename(columns = {'Time': 'Date'})
print(df_news.dtypes)

df_price['Date'] = pd.to_datetime(df_price['Date'], format='%m/%d/%Y')

Title              object
Tag                object
Date       datetime64[ns]
Content            object
dtype: object


In [4]:
df_price.sort_values('Date', inplace=True)

df_price.rename(columns={'Close/Last': 'Close'}, inplace=True)

df_price['Prev_change'] = df_price['Close'].pct_change()

df_price['Sma_5'] = df_price['Close'].rolling(window=5).mean() # rolling avg for the last 5 days 

df_price['Volatility'] = df_price['Close'].rolling(window=5).std() # volatility for the last 5 days 

df_price.head(10)

,Date,Close,Open,High,Low,Prev_change,Sma_5,Volatility
2517,2015-06-23,5160.09,5161.68,5163.41,5139.33,NaN,NaN,NaN
2516,2015-06-24,5122.41,5151.38,5164.36,5121.64,-0.007302,NaN,NaN
2515,2015-06-25,5112.19,5139.48,5141.71,5102.16,-0.001995,NaN,NaN
2514,2015-06-26,5080.51,5113.26,5121.47,5060.82,-0.006197,NaN,NaN
2513,2015-06-29,4958.47,5021.21,5051.01,4956.23,-0.024021,5086.734,77.123731
2512,2015-06-30,4986.87,5000.15,5008.76,4968.26,0.005728,5052.090,74.804682
2511,2015-07-01,5013.12,5029.05,5038.55,4994.46,0.005264,5030.232,64.358098
2510,2015-07-02,5009.21,5024.30,5027.47,4990.74,-0.000780,5009.636,45.198711
2509,2015-07-03,5009.21,0.00,0.00,0.00,0.000000,4995.376,23.086866
2508,2015-07-06,4991.94,4963.80,5020.71,4960.93,-0.003448,5002.070,11.808033


In [5]:
# Kursveränderung berechnen
df_price['Change'] = np.round(
            (df_price['Open'] - df_price['Close']) / df_price['Open'], 5
        )

# Label anhand der Kursveränderung
# 1 - Strong gain (>= +20%)
# 2 - Significant gain (+13% to <20%)
# 3 - Moderate gain (+7% to <13%)
# 4 - Stable gain (3% to <+7%)
# 5 - Stable neutral gain (1.5% to <+3%)
# 6 - Neutral (volatility) (-1.5% to <+1.5%)
# 7 - Stable neutral loss (3% to <-1.5%)
# 8 - Stable loss (-7% to <-3%)
# 9 - Moderate loss (-13% to <-7%)
# 10 - Significant loss (-20% to <-13%)
# 11 - Strong loss (< -20%)
conditions = [
    ((df_price['Change'] >= 0.015)), # 1
    ((df_price['Change'] >= 0) & (df_price['Change'] < 0.015)), # 2
    ((df_price['Change'] >= -0.015) & (df_price['Change'] < 0)), # 3
    ((df_price['Change'] < -0.015)) # 4
]
labels = [1, 2, 3, 4]
df_price['Label'] = np.select(conditions, labels)

df_price.head(10)
df_price.groupby(by=['Label']).count()

,Date,Close,Open,High,Low,Prev_change,Sma_5,Volatility,Change
Label,,,,,,,,,
1,185,185,185,185,185,185,185,185,185
2,955,955,955,955,955,954,951,951,955
3,1236,1236,1236,1236,1236,1236,1236,1236,1236
4,142,142,142,142,142,142,142,142,142


In [6]:
# decoupling, case-correction, pyrtial hyphen-resolution

def clean_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', str(text)).lower()
    words = text.split()
    
    tokens = []
    for w in words:
            tokens.extend(wordninja.split(w))
    
    tokens = [w for w in tokens if w not in stop_words and len(w) > 2]
    return tokens

df_news['Cleaned Content'] = df_news['Content'].apply(clean_text)

In [7]:
df_news['Weekday'] = df_news['Date'].dt.day_name()

df_news['Impact Date'] = np.where(
    df_news['Weekday'] != 'Friday',
    df_news['Date'] + pd.Timedelta(days=1),
    df_news['Date'] + pd.Timedelta(days=3)
)
df_news['Impact Date'] = pd.to_datetime(df_news['Impact Date'], format='%m/%d/%Y')

In [8]:
# Beide DataFrames mit Datumsspalten angleichen
#df_news['date'] = df_news['Time'].dt.date
#df_price['date'] = df_price['Date'].dt.date

# Merge
merged_df = pd.merge(df_news, 
                     df_price[['Date', 'Label', 'Change', 'Sma_5', 'Volatility', 'Prev_change']], 
                     left_on = 'Impact Date', right_on = 'Date', how='inner'
                    )
merged_df.head(10)

#merged_df.groupby(by=['Label']).count()

,Title,Tag,Date_x,Content,Cleaned Content,Weekday,Impact Date,Date_y,Label,Change,Sma_5,Volatility,Prev_change
0,"TSX Slightly Down, Books Weekly Gains",Stock Market,2023-07-16,"TSX Slightly Down, Books Weekly GainsUnited St...","[tsx, slightly, books, weekly, gains, united, ...",Sunday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299
1,US Budget Deficit Widens More than Expected in...,Government Budget Value,2023-07-14,US Budget Deficit Widens More than Expected in...,"[budget, deficit, widens, expected, june, unit...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299
2,Visa Hits 24-week High,stocks,2023-07-14,Visa Hits 24-week HighUnited States stocksVisa...,"[visa, hits, week, high, united, states, stock...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299
3,Amazon Hits 43-week High,stocks,2023-07-14,Amazon Hits 43-week HighUnited States stocksAm...,"[amazon, hits, week, high, united, states, sto...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299
4,10-Year Treasury Yield Falls for 4th Session,Government Bond 10Y,2023-07-14,10-Year Treasury Yield Falls for 4th SessionUn...,"[year, treasury, yield, falls, session, united...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299
5,DXY Approaches 100,Currency,2023-07-14,DXY Approaches 100United States CurrencyThe do...,"[approaches, united, states, currency, dollar,...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299
6,US Core PPI Rises Less than Expected,Core Producer Prices MoM,2023-07-14,US Core PPI Rises Less than ExpectedUnited Sta...,"[core, ppi, rises, less, expected, united, sta...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299
7,Progressive earnings below expectations at 0.5...,Earnings,2023-07-14,Progressive earnings below expectations at 0.5...,"[progressive, earnings, expectations, usd, uni...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299
8,PepsiCo earnings above expectations at 2.09 USD,Earnings,2023-07-14,PepsiCo earnings above expectations at 2.09 US...,"[pepsico, earnings, expectations, usd, united,...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299
9,US Budget Deficit Widens More than Expected in...,Government Budget Value,2023-07-14,US Budget Deficit Widens More than Expected in...,"[budget, deficit, widens, expected, june, unit...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299


In [9]:
def simplify_label(label):
    if label < 0:
        return 'negative'
    else:
        return 'positiv'

merged_df['Simple Label'] = merged_df['Change'].apply(simplify_label)
df_news = merged_df
df_news.head(10)


#df_news[df_news['Weekday'].isin(['Friday'])].sort_values(by = "Date", ascending=True)
# max 2023 - 07 - 14
# min 2015 - 06 - 23

,Title,Tag,Date_x,Content,Cleaned Content,Weekday,Impact Date,Date_y,Label,Change,Sma_5,Volatility,Prev_change,Simple Label
0,"TSX Slightly Down, Books Weekly Gains",Stock Market,2023-07-16,"TSX Slightly Down, Books Weekly GainsUnited St...","[tsx, slightly, books, weekly, gains, united, ...",Sunday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299,negative
1,US Budget Deficit Widens More than Expected in...,Government Budget Value,2023-07-14,US Budget Deficit Widens More than Expected in...,"[budget, deficit, widens, expected, june, unit...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299,negative
2,Visa Hits 24-week High,stocks,2023-07-14,Visa Hits 24-week HighUnited States stocksVisa...,"[visa, hits, week, high, united, states, stock...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299,negative
3,Amazon Hits 43-week High,stocks,2023-07-14,Amazon Hits 43-week HighUnited States stocksAm...,"[amazon, hits, week, high, united, states, sto...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299,negative
4,10-Year Treasury Yield Falls for 4th Session,Government Bond 10Y,2023-07-14,10-Year Treasury Yield Falls for 4th SessionUn...,"[year, treasury, yield, falls, session, united...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299,negative
5,DXY Approaches 100,Currency,2023-07-14,DXY Approaches 100United States CurrencyThe do...,"[approaches, united, states, currency, dollar,...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299,negative
6,US Core PPI Rises Less than Expected,Core Producer Prices MoM,2023-07-14,US Core PPI Rises Less than ExpectedUnited Sta...,"[core, ppi, rises, less, expected, united, sta...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299,negative
7,Progressive earnings below expectations at 0.5...,Earnings,2023-07-14,Progressive earnings below expectations at 0.5...,"[progressive, earnings, expectations, usd, uni...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299,negative
8,PepsiCo earnings above expectations at 2.09 USD,Earnings,2023-07-14,PepsiCo earnings above expectations at 2.09 US...,"[pepsico, earnings, expectations, usd, united,...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299,negative
9,US Budget Deficit Widens More than Expected in...,Government Budget Value,2023-07-14,US Budget Deficit Widens More than Expected in...,"[budget, deficit, widens, expected, june, unit...",Friday,2023-07-17,2023-07-17,3,-0.00672,14035.376,193.45833,0.009299,negative


In [10]:
merged_df.groupby(by=['Simple Label']).count()

,Title,Tag,Date_x,Content,Cleaned Content,Weekday,Impact Date,Date_y,Label,Change,Sma_5,Volatility,Prev_change
Simple Label,,,,,,,,,,,,,
negative,7585,7585,7585,7585,7585,7585,7585,7585,7585,7585,7585,7585,7585
positiv,6052,6052,6052,6052,6052,6052,6052,6052,6052,6052,6046,6046,6052


In [11]:
# Only text and labels
tokenized_df = merged_df